In [1]:
from model_training_ultils import ModelMemCalculation
from model_training_ultils import ModelEvaluationUltis
from data_repository import DataRepository
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [17]:
!conda list

Error while loading conda entry point: conda-libmamba-solver (libcrypto.so.1.1: cannot open shared object file: No such file or directory)
# packages in environment at /home/mrcong/anaconda3/envs/my-rdkit-env:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
_py-xgboost-mutex         2.0                       gpu_0    conda-forge
aom                       3.5.0                h27087fc_0    conda-forge
appdirs                   1.4.4              pyhd3eb1b0_0    anaconda
asttokens                 2.2.1              pyhd8ed1ab_0    conda-forge
backcall                  0.2.0              pyhd3eb1b0_0    anaconda
backports                 1.0                pyhd8ed1ab_3    conda-forge
backports.functools_lru_cache 1.6.4              pyhd8ed1ab_0    conda-forge
blas                      1.0                    openblas    anaco

# Ultilities to estimate the resource for training

- prun, a built-in python package is used to find the resource

In [2]:
mem_cal = ModelMemCalculation()
evaludation_tool = ModelEvaluationUltis()
data_repo = DataRepository("../.env")

## Memory record for Current models

In [3]:
init_mem = mem_cal.current_mem_usage()

Memory: 186512.0 kB


In [4]:
data_repo = DataRepository("../.env")
#Clean data
X_Train, y_train, X_validation, y_validation, X_test, y_test = data_repo.load_current_data(clean_data=True)

In [5]:
# Chuyển đổi danh sách labels thành mã số
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_validation = label_encoder.transform(y_validation)
y_test = label_encoder.transform(y_test)

In [6]:
data_mem = mem_cal.current_mem_usage()
print(f"Difference: {data_mem - init_mem}") 

Memory: 1356640.0 kB
Difference: 1170128.0


In [7]:
from sklearn.svm import SVC
svm_model_current = SVC(kernel='rbf', random_state=42, probability=True)
%prun svm_model_current.fit(X_Train, y_train)

         1246 function calls (1215 primitive calls) in 105.256 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1  105.025  105.025  105.025  105.025 {sklearn.svm._libsvm.fit}
        1    0.146    0.146    0.200    0.200 _methods.py:198(_var)
        5    0.082    0.016    0.082    0.016 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.001    0.001    0.201    0.201 {method 'var' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 class_weight.py:10(compute_class_weight)
        1    0.000    0.000    0.000    0.000 inspect.py:2426(_signature_from_callable)
        1    0.000    0.000  105.256  105.256 _base.py:147(fit)
        3    0.000    0.000    0.000    0.000 arraysetops.py:323(_unique1d)
        5    0.000    0.000    0.028    0.006 validation.py:629(check_array)
        1    0.000    0.000    0.000    0.000 validation.py:1207(check_random_state)
    37/22    0.000    0.000    0

In [8]:
after_training_svm_mem = mem_cal.current_mem_usage()
print(f"Difference: {after_training_svm_mem - data_mem}")

Memory: 1359192.0 kB
Difference: 2552.0


In [9]:
from sklearn.ensemble import RandomForestClassifier
rf_current = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=42)
%prun rf_current.fit(X_Train, y_train)

         251677 function calls (250256 primitive calls) in 4.084 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    3.834    0.038    3.834    0.038 {method 'build' of 'sklearn.tree._tree.DepthFirstTreeBuilder' objects}
      710    0.066    0.000    0.066    0.000 {built-in method numpy.asarray}
      501    0.020    0.000    0.020    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      301    0.009    0.000    0.023    0.000 validation.py:1207(check_random_state)
      201    0.009    0.000    0.009    0.000 {function SeedSequence.generate_state at 0x7f92f6231c60}
      601    0.008    0.000    0.028    0.000 inspect.py:2331(_signature_from_function)
     7819    0.007    0.000    0.012    0.000 inspect.py:2683(__init__)
      101    0.007    0.000    0.007    0.000 {method 'argsort' of 'numpy.ndarray' objects}
      100    0.006    0.000    3.924    0.039 _classes.py:176(fit)
      102    0.006    0.000    0.

In [10]:
after_training_rf_mem = mem_cal.current_mem_usage()
print(f"Difference: {after_training_rf_mem - after_training_svm_mem}")

Memory: 1363068.0 kB
Difference: 3876.0


In [11]:
from xgboost import XGBClassifier
xgboost_current = XGBClassifier(objective='binary:logistic', tree_method="gpu_hist")
%prun xgboost_current.fit(X_Train, y_train)

         11977 function calls (11960 primitive calls) in 8.782 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    8.117    0.081    8.120    0.081 core.py:1897(update)
        1    0.636    0.636    0.638    0.638 core.py:1402(_init)
        1    0.009    0.009    0.009    0.009 config.py:135(get_config)
        1    0.004    0.004    0.004    0.004 core.py:795(__del__)
        2    0.003    0.002    0.006    0.003 core.py:1528(__init__)
        1    0.003    0.003    0.003    0.003 core.py:1651(__del__)
        1    0.002    0.002    0.002    0.002 core.py:1656(__getstate__)
      100    0.001    0.000    0.001    0.000 core.py:1954(eval_set)
      303    0.001    0.000    0.001    0.000 core.py:1820(_get_feature_info)
        4    0.001    0.000    0.001    0.000 data.py:1039(_meta_from_numpy)
      100    0.000    0.000    0.002    0.000 callback.py:223(after_iteration)
      202    0.000    0.000    0.000    0

In [12]:
after_training_xgb_mem = mem_cal.current_mem_usage()
print(f"Difference: {after_training_xgb_mem - after_training_rf_mem}")

Memory: 1594268.0 kB
Difference: 231200.0


## Memory record for FFT models

In [3]:
initial_mem = mem_cal.current_mem_usage()

Memory: 186264.0 kB


In [4]:
data_repo = DataRepository("../.env")
#Clean data
train_ampls, y_train, validation_ampls, y_validation, test_ampls, y_test = data_repo.load_fft_data(clean_data=True)

In [5]:
print(f"Train dataset: X: {train_ampls.shape}, y: {y_train.shape}")
print(f"Validation dataset: X: {validation_ampls.shape}, y: {y_validation.shape}")
print(f"Test dataset: X: {test_ampls.shape}, y: {y_test.shape}")

Train dataset: X: (8188, 4501), y: (8188,)
Validation dataset: X: (1123, 4501), y: (1123,)
Test dataset: X: (2012, 4501), y: (2012,)


__Optimal k from validation set__
- k SVM = 454
- k RF = 12
- k XgBoost = 249

In [6]:
# Chuyển đổi danh sách labels thành mã số
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_validation = label_encoder.transform(y_validation)
y_test = label_encoder.transform(y_test)

def find_top_k_indices(amplitudes, k):
    # Get the indices of the top 50 elements
    top_k_indices = np.argsort(amplitudes)[-k:][::-1]
    highest_ampls = amplitudes[top_k_indices]
    return highest_ampls

def get_x_by_top_ampls(k, ampls):
    X = []
    for ampl in ampls:
        X.append(find_top_k_indices(amplitudes=ampl, k=k))
    return np.array(X)

k_svm = 1
k_rf = 1
k_xgb = 1

X_train_SVM = get_x_by_top_ampls(k=k_svm, ampls=train_ampls)
X_validation_SVM = get_x_by_top_ampls(k=k_svm, ampls=validation_ampls)
X_test_SVM = get_x_by_top_ampls(k=k_svm, ampls=test_ampls)

X_train_RF = get_x_by_top_ampls(k=k_rf, ampls=train_ampls)
X_validation_RF = get_x_by_top_ampls(k=k_rf, ampls=validation_ampls)
X_test_RF = get_x_by_top_ampls(k=k_rf, ampls=test_ampls)

X_train_XgBoost = get_x_by_top_ampls(k=k_xgb, ampls=train_ampls)
X_validation_XgBoost = get_x_by_top_ampls(k=k_xgb, ampls=validation_ampls)
X_test_XgBoost = get_x_by_top_ampls(k=k_xgb, ampls=test_ampls)

In [7]:
data_mem = mem_cal.current_mem_usage()
print(f"Difference: {data_mem - initial_mem}")

Memory: 705168.0 kB
Difference: 518904.0


In [8]:
from sklearn.svm import SVC
svm_fft = SVC(kernel='rbf', random_state=42, probability=True)
%prun svm_fft.fit(X_train_SVM, y_train)

         1246 function calls (1215 primitive calls) in 0.398 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.396    0.396    0.396    0.396 {sklearn.svm._libsvm.fit}
        1    0.000    0.000    0.000    0.000 class_weight.py:10(compute_class_weight)
        1    0.000    0.000    0.000    0.000 inspect.py:2426(_signature_from_callable)
        3    0.000    0.000    0.000    0.000 arraysetops.py:323(_unique1d)
        1    0.000    0.000    0.398    0.398 _base.py:147(fit)
        5    0.000    0.000    0.000    0.000 validation.py:629(check_array)
        1    0.000    0.000    0.000    0.000 validation.py:1207(check_random_state)
        5    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    0.000    0.000 {function SeedSequence.generate_state at 0x7fe094bddc60}
        1    0.000    0.000    0.000    0.000 _methods.py:198(_var)
    37/22    0.000 

In [9]:
after_training_svm_mem = mem_cal.current_mem_usage()
print(f"Difference: {after_training_svm_mem - data_mem}")

Memory: 707600.0 kB
Difference: 2432.0


In [10]:
# Tạo mô hình RF
from sklearn.ensemble import RandomForestClassifier
rf_fft = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=42)
%prun rf_fft.fit(X_train_RF, y_train)

         251677 function calls (250256 primitive calls) in 0.291 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.160    0.002    0.161    0.002 {method 'build' of 'sklearn.tree._tree.DepthFirstTreeBuilder' objects}
      201    0.008    0.000    0.008    0.000 {function SeedSequence.generate_state at 0x7fe094bddc60}
      601    0.007    0.000    0.026    0.000 inspect.py:2331(_signature_from_function)
     7819    0.007    0.000    0.012    0.000 inspect.py:2683(__init__)
      301    0.007    0.000    0.018    0.000 validation.py:1207(check_random_state)
      101    0.007    0.000    0.007    0.000 {method 'argsort' of 'numpy.ndarray' objects}
      102    0.006    0.000    0.006    0.000 {method 'sort' of 'numpy.ndarray' objects}
      203    0.004    0.000    0.020    0.000 arraysetops.py:323(_unique1d)
      100    0.003    0.000    0.229    0.002 _classes.py:176(fit)
      200    0.003    0.000    0.00

In [11]:
after_training_rf_mem = mem_cal.current_mem_usage()
print(f"Difference: {after_training_rf_mem - after_training_svm_mem}")

Memory: 711696.0 kB
Difference: 4096.0


In [12]:
from xgboost import XGBClassifier
xgboost_fft = XGBClassifier(objective='binary:logistic', tree_method="gpu_hist")
%prun xgboost_fft.fit(X_train_XgBoost, y_train)

         11977 function calls (11960 primitive calls) in 0.368 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.345    0.003    0.347    0.003 core.py:1897(update)
        1    0.009    0.009    0.009    0.009 config.py:135(get_config)
        2    0.003    0.002    0.006    0.003 core.py:1528(__init__)
        1    0.002    0.002    0.002    0.002 core.py:1656(__getstate__)
        1    0.001    0.001    0.002    0.002 core.py:1402(_init)
      100    0.001    0.000    0.001    0.000 core.py:1954(eval_set)
        1    0.001    0.001    0.001    0.001 core.py:1651(__del__)
      303    0.001    0.000    0.001    0.000 core.py:1820(_get_feature_info)
        4    0.000    0.000    0.000    0.000 data.py:1039(_meta_from_numpy)
      100    0.000    0.000    0.001    0.000 callback.py:223(after_iteration)
      101    0.000    0.000    0.002    0.000 core.py:2734(_validate_dmatrix_features)
      101    0.000   

In [13]:
after_training_xgboost_mem = mem_cal.current_mem_usage()
print(f"Difference: {after_training_xgboost_mem - after_training_rf_mem}")

Memory: 953408.0 kB
Difference: 241712.0
